In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec 



   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

C:\Users\masha\AppData\Local\Temp\ipykernel_8204\1080375821.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Score'] = positiveNegative


In [28]:

data = pd.read_csv(r"C:\Users\masha\git\NEEC\Reviews.csv")
print(data.head())

# Group by 'ProfileName' and count the occurrences
profile_counts = data['ProfileName'].value_counts()

# Get the profile names where the count is less than or equal to 50
valid_profiles = profile_counts[profile_counts >= 100].index


# Filter the DataFrame to keep only the rows where the 'ProfileName' is in the valid_profiles list
filtered_df = data[data['ProfileName'].isin(valid_profiles)]



   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [29]:
#change score value to "poitive" or "negative"
def classify_score(score):
    if score < 3:
        return 'negative'
    else:
        return 'positive'

In [30]:
#update score
actualScore = filtered_df['Score']
positiveNegative = actualScore.map(classify_score)
filtered_df['Score'] = positiveNegative

C:\Users\masha\AppData\Local\Temp\ipykernel_8204\1810857156.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Score'] = positiveNegative


In [13]:
filtered_df.shape
filtered_df.head

<bound method NDFrame.head of             Id   ProductId          UserId                       ProfileName  \
14          15  B001GVISJM  A2MUGFV2TDQ47K               Lynrie "Oh HELL no"   
46          47  B001EO5QW8   AQLL2R1PPR46X                     grumpyrainbow   
74          75  B001EPPI84  A3Q0IDQ03S0158                               Jen   
109        110  B001REEG6C   AY12DBB0U420B                     Gary Peterson   
136        137  B002SRYRE8  A198FU6P1BVUNZ                             Sarah   
...        ...         ...             ...                               ...   
568261  568262  B00374ZKQ0  A2GEZJHBV92EVR                      History buff   
568331  568332  B001BOAOLY  A36MP37DITBU6F  Enchanted  In  Dixie "Enchanted"   
568345  568346  B004BRECPW   AG3K28ZL7BR8M                               Jen   
568361  568362  B000LKVRQA  A1YUL9PCJR3JTY   O. Brown "Ms. O. Khannah-Brown"   
568367  568368  B000LKVRQA   AYB4ELCS5AM8P               John B. Goode "JBG"   

        H

In [25]:
#sorting data according to ProfileName
sorted_data=filtered_df.sort_values('ProfileName', axis=0, ascending=True)
sorted_data.head


157


(24545, 10)

In [26]:
#removing duplicate entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape


(16068, 10)

In [31]:
#keep entries where helfulness numerator is higher than denominator
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
print(final.shape)

(16068, 10)


In [32]:
#number of positive vs. negative reviews
final['Score'].value_counts()


Score
positive    14011
negative     2057
Name: count, dtype: int64

In [34]:
#Bag of Words
count_vect = CountVectorizer()
final = final.sort_values(by='Time')
final_counts = count_vect.fit_transform(final['Text'].values)


In [35]:
type(final_counts)

scipy.sparse._csr.csr_matrix

In [36]:
final_counts.get_shape


<bound method spmatrix.get_shape of <16068x27413 sparse matrix of type '<class 'numpy.int64'>'
	with 1066790 stored elements in Compressed Sparse Row format>>

In [38]:
#Text Preprocessing: Stemming, stop-word removal and Lemmatization
#find sentences containing HTML tags
import re

i=0;
for sent in final['Text'].values:
    if(len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
i += 1;


0
I don't know why anyone would ever use those little liquid food colors again after trying these pastes. The colors are more intense, and you can easily blend lovely shades for your icings. <p>Martha Stewart shows wonderful seasonal cookies from time to time on her TV show. A simple eggwhite-sugar icing on cookies in exciting colors makes such a wonderful table decoration as well as great dessert for parties, teas and other events. These paste colors are a must if you like to make frosted cookies or party cakes. <p>And these are especially fun if you have kids. A cookie-icing contest is a fun event for a birthday party, Christmas party or just a rainy afternoon.


In [51]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') # initilizes snowball stemmer


In [52]:
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
print(stop)
print('*************************************')
print(sno.stem('tasty'))

{'weren', "should've", "didn't", 'yourselves', 'my', 'mustn', 'over', 'ourselves', 'no', 'then', 'it', 'same', 'hers', 'mightn', 'does', "isn't", 'doing', 'both', 'any', "mustn't", 'of', 'once', "she's", 'his', 'those', 'll', 've', "won't", 'who', 'a', 'or', 'for', 'about', 're', 'am', 'when', 'nor', "it's", 't', "that'll", 'ours', 'which', 'too', 'ain', "don't", 'him', 'don', 'shan', 'is', 'your', 'you', 's', 'down', 'he', 'under', "wouldn't", 'so', 'we', 'by', 'through', 'where', 'has', 'needn', "doesn't", 'this', 'than', 'shouldn', 'were', 'such', 'be', 'her', 'as', 'theirs', "you'd", 'are', 'while', "wasn't", 'in', 'after', 'y', 'further', 'wasn', 'its', "shouldn't", 'm', 'do', 'had', 'again', 'they', 'our', 'and', 'if', 'before', 'now', "shan't", 'some', "aren't", 'these', 'from', 'during', 'just', 'more', 'below', 'was', 'she', 'only', 'between', 'into', 'but', 'hadn', "hadn't", 'himself', 'at', 'wouldn', 'on', 'whom', 'here', 'not', 'that', 'what', 'the', 'ma', 'me', 'off', 'the

In [53]:
i = 0
strl= ' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        
                        all_positive_words.append(s)
                    if (final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) 
                else:
                    continue
            else:
                continue
                
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    
    final_string.append(str1)
    i+=1

In [54]:
final['CleanedText']=final_string #adding a coln of CleanendText


In [55]:
#Bi-Grams and n-Grams
#analyze frequency distribution of words in positive/negative reviews
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most common positive words : ",freq_dist_positive.most_common(20))
print("Most common negative words : ",freq_dist_negative.most_common(20))


Most common positive words :  [(b'like', 8928), (b'flavor', 7809), (b'tast', 7770), (b'good', 6960), (b'tea', 6836), (b'use', 5916), (b'one', 5907), (b'great', 5155), (b'love', 4868), (b'product', 4758), (b'make', 4706), (b'tri', 4567), (b'food', 4146), (b'get', 3999), (b'coffe', 3781), (b'also', 3261), (b'would', 3260), (b'eat', 3213), (b'realli', 3178), (b'time', 2908)]
Most common negative words :  [(b'tast', 1409), (b'like', 1343), (b'product', 1044), (b'flavor', 869), (b'one', 862), (b'food', 754), (b'would', 729), (b'tri', 698), (b'good', 583), (b'eat', 556), (b'use', 541), (b'buy', 530), (b'even', 503), (b'get', 502), (b'dont', 476), (b'coffe', 467), (b'tea', 462), (b'order', 454), (b'dog', 451), (b'much', 432)]


In [58]:
#TF_IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [59]:
final_tf_idf.get_shape()


(16068, 422676)

In [60]:
features = tf_idf_vect.get_feature_names_out()
len(features)

422676

In [61]:
features[100000:100010]

array(['daughter reaction', 'daughter really', 'daughter reports',
       'daughter said', 'daughter seems', 'daughter she',
       'daughter simply', 'daughter specifically', 'daughter spit',
       'daughter started'], dtype=object)

In [62]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [63]:
import numpy as np
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [64]:
top_tfidf


,feature,tfidf
0,book,0.277714
1,read it,0.269347
2,would read,0.218802
3,this book,0.209923
4,read,0.155248
5,it,0.117672
6,either lost,0.109401
7,attic and,0.109401
8,somewhere probably,0.109401
9,at garage,0.109401


In [69]:
#word2vec
import gensim
list_of_sent = []
for sent in final['Text'].values:
    filtered_sentence = []
    # Perform text cleaning here
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_word in cleanpunc(w).split():
            if cleaned_word.isalpha():  # Corrected variable name
                filtered_sentence.append(cleaned_word.lower())  # Corrected variable name
    list_of_sent.append(filtered_sentence)

In [70]:
print(final['Text'].values[0])
print("**********************************************************")
print(list_of_sent[0])

I don't know why anyone would ever use those little liquid food colors again after trying these pastes. The colors are more intense, and you can easily blend lovely shades for your icings. <p>Martha Stewart shows wonderful seasonal cookies from time to time on her TV show. A simple eggwhite-sugar icing on cookies in exciting colors makes such a wonderful table decoration as well as great dessert for parties, teas and other events. These paste colors are a must if you like to make frosted cookies or party cakes. <p>And these are especially fun if you have kids. A cookie-icing contest is a fun event for a birthday party, Christmas party or just a rainy afternoon.
**********************************************************
['i', 'dont', 'know', 'why', 'anyone', 'would', 'ever', 'use', 'those', 'little', 'liquid', 'food', 'colors', 'again', 'after', 'trying', 'these', 'pastes', 'the', 'colors', 'are', 'more', 'intense', 'and', 'you', 'can', 'easily', 'blend', 'lovely', 'shades', 'for', 'you

In [71]:
w2v_model = gensim.models.Word2Vec(list_of_sent, min_count=5, vector_size=50, workers=4)
# Get the dimensions of word vectors
vector_size = w2v_model.vector_size
print(f"Word vector dimensions: {vector_size}")

Word vector dimensions: 50


In [72]:
words = list(w2v_model.wv.key_to_index)
print(len(words))

9326


In [73]:
similar_words = w2v_model.wv.most_similar('tasty')
print(similar_words)

[('yummy', 0.8432245254516602), ('satisfying', 0.8397105932235718), ('delicious', 0.825793981552124), ('filling', 0.8240694403648376), ('flavorful', 0.7353563904762268), ('versatile', 0.7291220426559448), ('moist', 0.7093454003334045), ('crunchy', 0.7065858840942383), ('nice', 0.7045647501945496), ('nutritious', 0.6815524101257324)]


In [74]:
similar_words = w2v_model.wv.most_similar('like')
print(similar_words)

[('prefer', 0.6275021433830261), ('mean', 0.6218447089195251), ('think', 0.6181060075759888), ('know', 0.5722044706344604), ('love', 0.5532277822494507), ('enjoy', 0.5434105396270752), ('miss', 0.5302862524986267), ('expect', 0.528601884841919), ('notice', 0.5199308395385742), ('gross', 0.5147050619125366)]


In [75]:
import numpy as np

# Assuming count_vect_feat is a numpy array
count_vect_feat = count_vect.get_feature_names_out()  # list of words in BoW
index_of_like = np.where(count_vect_feat == 'like')[0][0]  # Get the first index where 'like' occurs
print(index_of_like)

14612
